# Implikationen unterschiedlicher Risikomanagement-Strategien auf das Wert-/Risikoprofil einer Gesamtposition
© Thomas Robert Holy 2019
<br>
Version 0.3.6
<br><br>
Visit me on GitHub: https://github.com/trh0ly
## Grundlegende Einstellungen:
Zunächst müssen die notwendigen Pakete (auch Module) importiert werden, damit auf diese zugegriffen werden kann. 

In [ ]:
import pandas as pd # Programmbibliothek die Hilfsmittel für die Verwaltung von Daten und deren Analyse anbietet
import scipy.stats as stats # SciPy ist ein Python-basiertes Ökosystem für Open-Source-Software für Mathematik, Naturwissenschaften und Ingenieurwissenschaften
from scipy.stats import rankdata, norm  
from scipy import array, linalg, dot
import random # Dieses modul wird verwendet um Zufallszahlen zu ziehen
import numpy as np # Programmbibliothek die eine einfache Handhabung von Vektoren, Matrizen oder generell großen mehrdimensionalen Arrays ermöglicht
import math # Dieses Modul wird verwendet um Skalardaten zu berechnen, z. B. trigonometrische Berechnungen.
import operator # Programmbibliothek, welche die Ausgaben übersichtlicher gestaltet
import matplotlib.pyplot as plt # Programmbibliothek die es erlaubt mathematische Darstellungen aller Art anzufertigen
import matplotlib.patches as mpatches
import datetime as dt # Das datetime-Modul stellt Klassen bereit, mit denen Datums- und Uhrzeitangaben auf einfache und komplexe Weise bearbeitet werden können
import random # Dieses Modul implementiert Pseudozufallszahlengeneratoren für verschiedene Verteilungen.
import riskmeasure_module as rm # Dieses "Modul" vereinfacht die Berechnung der Risikomaße

Anschließend werden Einstellungen definiert, die die Formatierung der Ausgaben betreffen.
Hierfür wird das Modul `operator` genutzt.
Außerdem wird die Größe der Grafiken modifiziert, welche später angezeigt werden sollen.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

##########################################################################
#-------------------------------------------------------------------------

SCREEN_WIDTH = 90
centered = operator.methodcaller('center', SCREEN_WIDTH) 
pd.set_option('display.width', 125) 
plt.rcParams["figure.figsize"] = 15,12.5 

#-------------------------------------------------------------------------
##########################################################################

## Datensätze einlesen und manipulieren:
Nun werden Datensätze eingelesen und manipuliert.
Da Jupyter Notebook leider Eingaben nicht zeilenweise einlesen kann, werden die Datensätze manuell definiert und anschließend zum Array "dateinamen" hinzufügt.
Standardmäßig werden fünf Datensätze (example1, ..., example5) definiert und im Array "dateinamen" gespeichert.
<br>
Anschließend wird aus jedem eingelesen Datensatz der Aktienkurs zum jeweiligen Tag extrahiert werden. 
Dieser Schritt wird automatisiert, indem zunächst die leere Liste "kurse" anlegt wird und anschließend von jedem sich in der Liste "dateinamen" befindenden Eintrag die jeweiligen Spalten "Date" und "Adj Close" eingelesen werden.
Dabei werden die verschiedenen im Datensatz vorhanden Spalten mit jedem Komma separiert und Punkte werden als Zeichen für die Dezimaltrennung interpretiert. Anschließend werden die so extrahierten Daten zum Array "kurse" hinzugefügt.
<br>
Danach wird das Modul `datetime` genutzt, um die Datumsspalte des jeweiligen Datensatzes bearbeitbar zu machen.
Zudem wird dem Programm mitgeteilt, dass die Einträge der Spalte "Adj Close" numerisch sind und mit ihnen gerechnet werden kann. Kommt es dabei zu Fehlern werden die entsprechende Werte als NaN-Werte behandelt.
<br><br>
Hinweis: An dieser Stelle können alternativ "richtige" Datensätze eingelesen werden, indem z.B. "example1" in BAS.DE usw. umbenannt werden, sofern die entsprechenden Datensätze im Home-Verzeichnis hochgeladen wurden.

In [ ]:
##########################################################################
#-------------------------------------------------------------------------
trading_days = 253

datensatz1 = 'example1'
datensatz2 = 'example2'
datensatz3 = 'example3'
datensatz4 = 'example4'
datensatz5 = 'example5'

dateinamen = [datensatz1,datensatz2,datensatz3,datensatz4,datensatz5]

#-------------------------------------------------------------------------
##########################################################################

kurse = []
for eintrag in dateinamen:
    kurs = pd.read_csv(str(eintrag) + '.csv',
                decimal='.',
                usecols=['Date','Adj Close'])
    kurse.append(kurs) 

for eintrag in kurse:
    eintrag['Date'] = pd.to_datetime(eintrag['Date']) 
    eintrag['Adj Close'] = pd.to_numeric(eintrag['Adj Close'], errors='coerce') 

Nun werden zwei verschiedene DataFrames erzeugt, wobei "kurschart_0" die Basis für das Sharpe-Portfolio und "kurschart_1" die Basis für die Naive Diversifikation darstellt.
Beide beinhalten die täglichen aus den eingelesenen Aktienkursen berechneten Renditen, wobei für "kurschart_1" direkt die Portfolio-Rendite mittels Naiver Diversifikation ermittelt wird.
Beide DataFrames sind auf die Handelstage eines Jahres beschränkt.

In [ ]:
kurschart_0 = pd.DataFrame()
kurschart_1 = pd.DataFrame()

zaehler = 0
for eintrag in kurse:
    x = dateinamen[zaehler]
    kurschart_0['Aktienkurs ' + str(x)] = eintrag['Adj Close']
    kurschart_1['Aktienkurs ' + str(x)] = eintrag['Adj Close']    
    zaehler += 1

kurschart_0 = kurschart_0[:(trading_days - 1)]
kurschart_0 = kurschart_0.pct_change()
kurschart_1 = kurschart_1[:(trading_days - 1)]
kurschart_1 = kurschart_1.pct_change()

#-------------------------------------------------------------------------
# Naive Diversifikation 

kurschart_1['PF-Rendite'] = (kurschart_1.sum(axis = 1, skipna = True) / len(dateinamen))

## Bestimmung des optimalen Sharpe-Portfolios mittels Monte-Carlo-Simulation
In diesem Abschnitt werden zunächst die jährlichen Renditen und Kovarianzen der einzelnen Assets bestimmt.
Anschließend wird auf Basis dieser Informationen eine Monte-Carlo-Siumulation durchgeführt durch welche das optimale Sharpe-Portfolio bestimmt wird.
Hierfür werden n Zufallszahlen gezogen (n = Anzahl der Assets im Portfolio), welche anschließend so nomiert werden, dass sie in Summe Eins ergeben aber die Relationen untereinander erhalten bleiben.
Im Anschluss wird die jeweilige jährliche Renditen der Asserts mit dem jeweiligen zuvor ermittelten Gewicht multipliziert, sodass sich eine Portfolio-Rendite ergibt.
Gleichermaßen wird die dazugehörige Volatilität bestimmt, sodass daraufhin der Sharpe-Ratio ermittelt werden kann.
Die so berechneten Größen werden jeweils in einer Liste gespeichert und in den DataFrame "PF_DataFrame" überführt, aus welchem sowohl das optimale Sharpe- als auch das Minimum-Varianz-Portfolio bestimmt wird.
Zum Schluss wird eine Grafik generiert, welche den effizienten Rand darstellt.
<br><br>
Hinweis: Dieser Programmteil entstand in Anlehngung an "Efficient Frontier & Portfolio Optimization with Python" von Bernard Brenyah (https://github.com/PyDataBlog/Python-for-Data-Science/blob/master/Tutorials/Efficient%20Frontier%20with%20Sharpe%20Ratio.py) **

In [ ]:
returns_annual = kurschart_0.mean() * trading_days
cov_daily = kurschart_0.cov()
cov_annual = cov_daily * math.sqrt(trading_days)

#-------------------------------------------------------------------------

PF_return_list = []
PF_volatility_list = []
sharpe_ratio_list = []
asset_weights_list = []

n = 25000
num_of_assets = len(dateinamen)

for i in range(0,n):
    random_weights = np.random.random(num_of_assets)
    PF_weights = random_weights / np.sum(random_weights)
    
    PF_returns = np.dot(PF_weights, returns_annual)
    PF_volatility = np.sqrt(np.dot(PF_weights.T, np.dot(cov_annual, PF_weights)))
    sharpe_ratio = PF_returns / PF_volatility

    asset_weights_list.append(PF_weights)
    PF_return_list.append(PF_returns)
    PF_volatility_list.append(PF_volatility)
    sharpe_ratio_list.append(sharpe_ratio)

#-------------------------------------------------------------------------

PF_DataFrame = pd.DataFrame({'PF_Return': PF_return_list, 'PF_Volatility': PF_volatility_list, 'PF_Sharpe_Ratio': sharpe_ratio_list})
for counter, dateiname in enumerate(dateinamen):
    PF_DataFrame[str(dateiname) + ' Weight'] = [weight[counter] for weight in asset_weights_list]

min_volatility = PF_DataFrame['PF_Volatility'].min()
max_sharpe_ratio = PF_DataFrame['PF_Sharpe_Ratio'].max()
Sharpe_PF = PF_DataFrame.loc[PF_DataFrame['PF_Sharpe_Ratio'] == max_sharpe_ratio]
Min_variance_PF = PF_DataFrame.loc[PF_DataFrame['PF_Volatility'] == min_volatility]

#-------------------------------------------------------------------------

plt.scatter(PF_volatility_list, PF_return_list, c = sharpe_ratio_list, marker='o', cmap='coolwarm', edgecolors='black')
plt.colorbar(label='Sharpe Ratio')
plt.scatter(x = Sharpe_PF['PF_Volatility'], y = Sharpe_PF['PF_Return'], color='red', marker='v', edgecolors='black', s=300)
plt.scatter(x = Min_variance_PF['PF_Volatility'], y = Min_variance_PF['PF_Return'], color='blue', edgecolors='black', marker='v', s=300)
plt.grid()
plt.ylabel('Erwartete Rendite')
plt.title('Effizienter Rand und optimale Portfolios')
plt.xlabel('Volatilität (Standardabweichung)')
plt.show()

#-------------------------------------------------------------------------

print('#' + SCREEN_WIDTH * '-' + '#')
print('Minimum-Varianz-Portfolio: \n' + str(Min_variance_PF.T))
print('#' + SCREEN_WIDTH * '-' + '#')
print('Optimales Sharpe-Portfolio: \n' + str(Sharpe_PF.T))
print('#' + SCREEN_WIDTH * '-' + '#')

## Optimales Sharpe-Portfolio und Datenbereinigung
In diesem Schritt werden die durch die Monte-Carlo-Simulation ermittelten optimalen Gewichte des Sharpe-Portfolios auf den DataFrame "kurschart_0" angewendet, sodass im Anschluss die Portfolio-Rendite errechnet werden kann.
Danach werden die Portfolio-Renditen in einer Liste gespeichert und bereinigt.

In [ ]:
#-------------------------------------------------------------------------
# Optimales Sharpe-Portfolio

Sharpe_PF = Sharpe_PF.values.tolist()
Sharpe_PF = Sharpe_PF[0][3::]

kurschart_0 = kurschart_0.multiply(Sharpe_PF, axis = 1)
kurschart_0['PF-Rendite'] = kurschart_0.sum(axis = 1, skipna = True)

#-------------------------------------------------------------------------
# Datenbereinigung

sharpe_values_PF = kurschart_0['PF-Rendite'].values.tolist()
sharpe_values_PF = np.array(sharpe_values_PF)
sharpe_values_PF = sharpe_values_PF[np.logical_not(np.isnan(sharpe_values_PF))]

#----------------------------------

naiv_values_PF = kurschart_1['PF-Rendite'].values.tolist()
naiv_values_PF = np.array(naiv_values_PF)
naiv_values_PF = naiv_values_PF[np.logical_not(np.isnan(naiv_values_PF))]

## Auswertung des Sharpe-Portfolios
### Funktionen Definieren und Verteilungsfunktionen plotten
In diesem Abschnitt werden zunächst Mittelwert und Standardabweichung des Sharpe-Portfolios berechnet und ausgegeben. 
Anschließend werden Funktionen definiert, welche das Plotten der Verteilungsfunktionen für die historische Simulation und die Varianz-Kovarianz-Methode und die Ermittlung der Risikomaße auf Basis des jeweiligen Simulationsverfahrens vereinfachen.

In [ ]:
print((SCREEN_WIDTH + 2) * '#')
print('|' + centered('Optimales PF nach Sharpe') + '| ')
print((SCREEN_WIDTH + 2) * '#')

#-------------------------------------------------------------------------

mu_sharpe_PF = np.mean(sharpe_values_PF)
std_sharpe_PF = np.std(sharpe_values_PF)

print('#' + SCREEN_WIDTH * '-' + '#')
print('|' + centered('[INFO] Die Porfolio-Rendite hat einen Erwartunswert i.H.v. ' + str(mu_sharpe_PF) + '.') + '| ')
print('#' + SCREEN_WIDTH * '-' + '#')
print('|' + centered('[INFO] Das Porfolio hat eine Standardabweichung i.H.v. ' + str(std_sharpe_PF) + '.') + '| ')
print('#' + SCREEN_WIDTH * '-' + '#')

mini_values_PF = min(min(naiv_values_PF), min(sharpe_values_PF))
maxi_values_PF = max(max(naiv_values_PF), max(sharpe_values_PF)) 

bins = len(naiv_values_PF)

#-------------------------------------------------------------------------
# Historische Simulation - Plot

def hist_sim(values, bins):
    H, X1 = np.histogram(values, bins, density=True)
    dx = X1[1] - X1[0]
    F1 = np.cumsum(H) * dx 
    plt.plot(X1[1:], F1) 
    
hist_sim(sharpe_values_PF, bins)

#-------------------------------------------------------------------------
# Varianz-Kovarianz-Methode - Plot

def var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu, std):
    global var_covar_results
    
    array = np.array(np.arange(0.0001, 1, 0.0001))
    var_covar_results = stats.norm.ppf(array, mu, std)

    var_covar_range = np.linspace(mini_values_PF, maxi_values_PF, bins)
    plt.plot(var_covar_range, stats.norm.cdf(var_covar_range, mu, std))
    
var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_sharpe_PF, std_sharpe_PF)

#-------------------------------------------------------------------------
# Restliche Einstellungen für die Grafik

def easy_plot():
    plt.xlabel('Rendite') 
    plt.ylabel('Wahrscheinlichkeit') 
    blue_patch = mpatches.Patch(color='blue', label='Historische Simulation') 
    orange_patch = mpatches.Patch(color='orange', label='Varianz-Kovarianzmethode') 
    plt.legend(handles=[orange_patch, blue_patch]) 
    plt.title('Verteilungsfunktion: Historische Simulation versus Varianz-Kovarianz-Methode')
    plt.grid() 
    plt.axhline(0, color='black') 
    plt.axvline(0, color='black') 
    plt.show() 
easy_plot() 

#-------------------------------------------------------------------------
# Risikomessung - Historische Simulation

RM_list = []
def risk(values, alpha, gamma):
    global RM_list
    
    alle_RM =  rm.risk_measure()
    alle_RM.get_all(values, alpha, gamma)
    VaR, CVaR, Power_Risk, Power_EW = alle_RM.all
    
    RM_list.append([VaR, CVaR, Power_Risk, Power_EW])

    print('|' + centered('Der VaR beträgt: ' + str(VaR) + '.') + '| ')
    print('|' + centered('Der CVaR beträgt: ' + str(CVaR) + '.') + '| ')
    print('#' + SCREEN_WIDTH * '-' + '#')
    print('|' + centered('Power-Spektrales Risikomaß:') + '| ')
    print('|' + centered('Der Erwartungswert beträgt: ' + str(Power_EW) + '.') + '| ')
    print('|' + centered('Das Risiko beträgt: ' + str(Power_Risk) + '.') + '| ')

### Risikomaße schätzen - Parameterfestlegung und Aufruf der Funktionen

In [ ]:
##########################################################################
#-------------------------------------------------------------------------

alpha = 0.1
gamma = 0.5

#-------------------------------------------------------------------------
##########################################################################

print((SCREEN_WIDTH + 2) * '#')
print('|' + centered('Sharpe: Risikomessung - Historische Simulation') + '| ')
print((SCREEN_WIDTH + 2) * '#')
risk(sharpe_values_PF, alpha, gamma)

#-------------------------------------------------------------------------
# Risikomessung - Varianz-Kovarianz-Methode

print((SCREEN_WIDTH + 2) * '#')
print('|' + centered('Sharpe: Risikomessung - Varianz-Kovarianz-Methode') + '| ')
print((SCREEN_WIDTH + 2) * '#')

risk(var_covar_results, alpha, gamma)
print('#' + SCREEN_WIDTH * '-' + '#')

## Auswertung des naiv diversifizierten Portfolios
Da die Funktionen zum Plotten der Verteilungsfunktionen und zur Risikomessung im vorherigen Schritt bereits definiert wurden, müssen diese hier nur noch mit den entsprechenden Daten der naiven Diversifikation aufgerufen werden.

In [ ]:
print((SCREEN_WIDTH + 2) * '#')
print('|' + centered('Naive Diversifikation') + '| ')
print((SCREEN_WIDTH + 2) * '#')

#-------------------------------------------------------------------------

mu_naiv_PF = np.mean(naiv_values_PF)
std_naiv_PF = np.std(naiv_values_PF)

print('#' + SCREEN_WIDTH * '-' + '#')
print('|' + centered('[INFO] Die Porfolio-Rendite hat einen Erwartunswert i.H.v. ' + str(mu_naiv_PF) + '.') + '| ')
print('#' + SCREEN_WIDTH * '-' + '#')
print('|' + centered('[INFO] Das Porfolio hat somit eine Standardabweichung i.H.v. ' + str(std_naiv_PF) + '.') + '| ')
print('#' + SCREEN_WIDTH * '-' + '#')

#-------------------------------------------------------------------------
# Historische Simulation - Plot

hist_sim(naiv_values_PF, bins)

#-------------------------------------------------------------------------
# Varianz-Kovarianz-Methode - Plot

var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_naiv_PF, std_naiv_PF)

#-------------------------------------------------------------------------
# Restliche Einstellungen für die Grafik

easy_plot() 

#-------------------------------------------------------------------------
# Risikomessung - Historische Simulation

print((SCREEN_WIDTH + 2) * '#')
print('|' + centered('Sharpe: Risikomessung - Historische Simulation') + '| ')
print((SCREEN_WIDTH + 2) * '#')

risk(naiv_values_PF, alpha, gamma)

#-------------------------------------------------------------------------
# Risikomessung - Varianz-Kovarianz-Methode

print((SCREEN_WIDTH + 2) * '#')
print('|' + centered('Sharpe: Risikomessung - Varianz-Kovarianz-Methode') + '| ')
print((SCREEN_WIDTH + 2) * '#')

risk(var_covar_results, alpha, gamma)
print('#' + SCREEN_WIDTH * '-' + '#')

## Gegenüberstellung der Verteilungsfunktionen und Risikomaße
Um die Verteilungsfunktionen und die berechneten Risikomaße besser vergleichen zu können, werden diese hier noch einmal zusammengetragen.

In [ ]:
#-------------------------------------------------------------------------
#  Sharpe-Portfolio: Historische Simulation - Plot (Blau)

hist_sim(sharpe_values_PF, bins)

#-------------------------------------------------------------------------
# Sharpe-Portfolio: Varianz-Kovarianz-Methode - Plot (Orange)

var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_sharpe_PF, std_sharpe_PF)

#-------------------------------------------------------------------------
# Naive Diversifikation: Historische Simulation - Plot (Grün)

hist_sim(naiv_values_PF, bins)

#-------------------------------------------------------------------------
# Naive Diversifikation: Varianz-Kovarianz-Methode - Plot (Rot)

var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_naiv_PF, std_naiv_PF)

#-------------------------------------------------------------------------
# Restliche Einstellungen für die Grafik

plt.xlabel('Rendite') 
plt.ylabel('Wahrscheinlichkeit') 
blue_patch = mpatches.Patch(color='blue', label='Sharpe - Historische Simulation') 
orange_patch = mpatches.Patch(color='orange', label='Sharpe - Varianz-Kovarianzmethode') 
green_patch = mpatches.Patch(color='green', label='Naiv - Historische Simulation') 
red_patch = mpatches.Patch(color='red', label='Naiv - Varianz-Kovarianzmethode')     
plt.legend(handles=[orange_patch, blue_patch, green_patch, red_patch]) 
plt.title('Verteilungsfunktion: Sharpe/Naiv - Historische Simulation versus Varianz-Kovarianz-Methode')
plt.grid() 
plt.axhline(0, color='black') 
plt.axvline(0, color='black') 
plt.show() 

#-------------------------------------------------------------------------
# DataFrame Risikomessung: Übersicht

RM_DataFrame = pd.DataFrame()
RM_DataFrame['Index'] = ['VaR', 'CVaR', 'P-SRM (Risk)', 'P-SRM (EW)']
RM_DataFrame['Sharpe + Historisch'] = RM_list[0]
RM_DataFrame['Naiv + Historisch'] = RM_list[2]
RM_DataFrame['Sharpe + VarKoVar'] = RM_list[1]
RM_DataFrame['Naiv + VarKoVar'] = RM_list[3]
RM_DataFrame = RM_DataFrame.set_index('Index')

print('#' + SCREEN_WIDTH * '-' + '#')
print('|' + centered('Sharpe Portfolio versus Naive Diversifikation: Risikomessung') + '| ')
print('#' + SCREEN_WIDTH * '-' + '#')
print(RM_DataFrame)
print('#' + SCREEN_WIDTH * '-' + '#')

___

** Hinweis zur Verwendung des modizifierten Quellcodes von Bernard Brenyah zur Bestimmung des Sharpe Portfolios mittels Monte-Carlo Simulation:

MIT License

Copyright (c) 2018 Bernard Brenyah

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.